In [1]:
from mmcv import Config
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector
from mmdet.datasets import (build_dataloader, build_dataset, replace_ImageToTensor)

/opt/conda/lib/python3.7/site-packages/mmcv/utils/registry.py:250: UserWarning: The old API of register_module(module, force=False) is deprecated and will be removed, please use the new API register_module(name=None, force=False, module=None) instead.
  'The old API of register_module(module, force=False) '


In [2]:
classes = ("UNKNOWN", "General trash", "Paper", "Paper pack", "Metal", "Glass", 
           "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing")
# config file 들고오기
cfg = Config.fromfile('/opt/ml/code/UniverseNet/configs/universenet/universenet101_2008d_fp16_4x4_mstrain_480_960_20e_coco.py')

PREFIX = '/opt/ml/input/data/'

# dataset 바꾸기
cfg.data.train.classes = classes
cfg.data.train.img_prefix = PREFIX
cfg.data.train.ann_file = PREFIX + 'train_all.json'

cfg.data.val.classes = classes
cfg.data.val.img_prefix = PREFIX
cfg.data.val.ann_file = PREFIX + 'val.json'

cfg.data.test.classes = classes
cfg.data.test.img_prefix = PREFIX
cfg.data.test.ann_file = PREFIX + 'test.json'

cfg.model.backbone.norm_cfg.type = 'BN'
# cfg.runner.type = "EpochBasedRunnerAmp"

# augmentation
img_norm_cfg = dict(mean=[123.675, 116.28, 103.53], std=[58.395, 57.12, 57.375], to_rgb=True)
albu_train_transforms = [
    dict(
        type="Cutout",
        num_holes=30,
        max_h_size=30,
        max_w_size=30,
        fill_value=img_norm_cfg["mean"][::-1],
        p=0.1,
    ),
    # albu_example
    dict(
        type='RandomBrightnessContrast',
        brightness_limit=[0.1, 0.3],
        contrast_limit=[0.1, 0.3],
        p=0.2),
    dict(
        type='OneOf',
        transforms=[
            dict(
                type='RGBShift',
                r_shift_limit=10,
                g_shift_limit=10,
                b_shift_limit=10,
                p=1.0),
            dict(
                type='HueSaturationValue',
                hue_shift_limit=20,
                sat_shift_limit=30,
                val_shift_limit=20,
                p=1.0),
            dict(type="RandomGamma"),
            dict(type="CLAHE"),
        ],
        p=0.1),
    dict(type='JpegCompression', quality_lower=85, quality_upper=95, p=0.2),
    dict(type='ChannelShuffle', p=0.1),
    dict(
        type='OneOf',
        transforms=[
            dict(type='Blur', blur_limit=3, p=1.0),
            dict(type='MedianBlur', blur_limit=3, p=1.0),
            dict(type="MotionBlur"),
            dict(type="GaussNoise"),
            dict(type="ImageCompression", quality_lower=75),
        ],
        p=0.1),
    dict(type='RandomRotate90', p=0.2),
]
cfg.data.train.pipeline.insert(4, dict(
        type='Albu',
        transforms=albu_train_transforms,
        bbox_params=dict(
            type='BboxParams',
            format='pascal_voc',
            label_fields=['gt_labels'],
            min_visibility=0.0,
            filter_lost_elements=True),
        keymap={
            'img': 'image',
            'gt_masks': 'masks',
            'gt_bboxes': 'bboxes'
        },
        update_pad_shape=False,
        skip_img_without_anno=True))

cfg.data.samples_per_gpu = 8
cfg.data.workers_per_gpu = 1

cfg.seed=2020
cfg.gpu_ids = [0]
cfg.work_dir = '/opt/ml/code/UniverseNet/work_dirs/universenet101_2008d_fp16_4x4_mstrain_480_960_20e_coco'

cfg.model.bbox_head.num_classes = 11

cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)

cfg.load_from = 'universenet101_2008d_fp16_4x4_mstrain_480_960_20e_coco_20201023_epoch_20-3e0d236a.pth' # pretrained
cfg.log_config = dict(  # config to register logger hook
    interval=100,  # Interval to print the log
    hooks=[
        dict(type='TensorboardLoggerHook'),  # The Tensorboard logger is also supported
        dict(type='TextLoggerHook')
    ])
print(f'Config:\n{cfg.pretty_text}')

Config:
model = dict(
    type='GFL',
    pretrained='open-mmlab://res2net101_v1d_26w_4s',
    backbone=dict(
        type='Res2Net',
        depth=101,
        scales=4,
        base_width=26,
        num_stages=4,
        out_indices=(0, 1, 2, 3),
        frozen_stages=1,
        norm_cfg=dict(type='BN', requires_grad=True),
        norm_eval=False,
        style='pytorch',
        dcn=dict(type='DCN', deform_groups=1, fallback_on_stride=False),
        stage_with_dcn=(False, True, True, True)),
    neck=[
        dict(
            type='FPN',
            in_channels=[256, 512, 1024, 2048],
            out_channels=256,
            start_level=1,
            add_extra_convs='on_output',
            num_outs=5),
        dict(
            type='SEPC',
            out_channels=256,
            stacked_convs=4,
            pconv_deform=True,
            lcconv_deform=True,
            ibn=True,
            pnorm_eval=False,
            lcnorm_eval=False,
            lcconv_padding=1)
   

In [3]:
model = build_detector(cfg.model)

2021-05-20 01:38:51,177 - mmdet - INFO - load model from: open-mmlab://res2net101_v1d_26w_4s
2021-05-20 01:38:51,178 - mmdet - INFO - Use load_from_openmmlab loader
2021-05-20 01:38:51,411 - mmdet - WARNING - The model and loaded state dict do not match exactly

unexpected key in source state_dict: fc.weight, fc.bias

missing keys in source state_dict: layer2.0.convs.0.conv_offset.weight, layer2.0.convs.0.conv_offset.bias, layer2.0.convs.1.conv_offset.weight, layer2.0.convs.1.conv_offset.bias, layer2.0.convs.2.conv_offset.weight, layer2.0.convs.2.conv_offset.bias, layer2.1.convs.0.conv_offset.weight, layer2.1.convs.0.conv_offset.bias, layer2.1.convs.1.conv_offset.weight, layer2.1.convs.1.conv_offset.bias, layer2.1.convs.2.conv_offset.weight, layer2.1.convs.2.conv_offset.bias, layer2.2.convs.0.conv_offset.weight, layer2.2.convs.0.conv_offset.bias, layer2.2.convs.1.conv_offset.weight, layer2.2.convs.1.conv_offset.bias, layer2.2.convs.2.conv_offset.weight, layer2.2.convs.2.conv_offset.bia

In [4]:
datasets = [build_dataset(cfg.data.train)]

loading annotations into memory...
Done (t=4.80s)
creating index...
index created!


In [5]:
train_detector(model, datasets[0], cfg, distributed=False, validate=True)

loading annotations into memory...
Done (t=0.92s)
creating index...


2021-05-20 01:39:01,999 - mmdet - INFO - load checkpoint from universenet101_2008d_fp16_4x4_mstrain_480_960_20e_coco_20201023_epoch_20-3e0d236a.pth
2021-05-20 01:39:02,000 - mmdet - INFO - Use load_from_local loader


index created!


2021-05-20 01:39:02,326 - mmdet - WARNING - The model and loaded state dict do not match exactly

size mismatch for bbox_head.gfl_cls.weight: copying a param with shape torch.Size([80, 256, 3, 3]) from checkpoint, the shape in current model is torch.Size([11, 256, 3, 3]).
size mismatch for bbox_head.gfl_cls.bias: copying a param with shape torch.Size([80]) from checkpoint, the shape in current model is torch.Size([11]).
2021-05-20 01:39:02,329 - mmdet - INFO - Start running, host: root@03064abcf195, work_dir: /opt/ml/code/UniverseNet/work_dirs/universenet101_2008d_fp16_4x4_mstrain_480_960_20e_coco
2021-05-20 01:39:02,332 - mmdet - INFO - workflow: [('train', 1)], max: 20 epochs
/opt/ml/code/UniverseNet/mmdet/core/evaluation/eval_hooks.py:111: UserWarning: runner.meta is None. Creating a empty one.
  warnings.warn('runner.meta is None. Creating a empty one.')
2021-05-20 01:42:34,726 - mmdet - INFO - Epoch [1][100/409]	lr: 9.990e-04, eta: 4:45:46, time: 2.122, data_time: 0.041, memory: 2

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 655/655, 10.1 task/s, elapsed: 65s, ETA:     0s

...

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 655/655, 10.6 task/s, elapsed: 62s, ETA:     0s

2021-05-20 06:52:02,485 - mmdet - INFO - Evaluating bbox...


Loading and preparing results...
DONE (t=0.74s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=7.08s).
Accumulating evaluation results...


2021-05-20 06:52:11,676 - mmdet - INFO - Now best checkpoint is epoch_20.pth.Best bbox_mAP is 0.6610


DONE (t=1.26s).
Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.661
Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=1000 ] = 0.775
Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=1000 ] = 0.697
Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.406
Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.691
Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=1000 ] = 0.839
Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.753
Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=300 ] = 0.753
Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=1000 ] = 0.753
Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.523
Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.802
Average Recall     (AR) @[ IoU=0.50:0.95 | area= large | maxDets=1000 ] = 0.899
